## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-09-22-52-57 +0000,nyt,"Gavin Newsom on Democrats, 2028 and His Fruit-...",https://www.nytimes.com/2026/01/09/us/politics...
1,2026-01-09-22-52-27 +0000,nyt,"Hessy Levinsons Taft, Jewish Baby on Cover of ...",https://www.nytimes.com/2026/01/09/world/europ...
2,2026-01-09-22-51-00 +0000,wsj,Minnesota Prosecutors Seek Evidence From Fatal...,https://www.wsj.com/us-news/minnesota-official...
3,2026-01-09-22-51-00 +0000,wsj,How Minnesota Became the Flashpoint in Trump’s...,https://www.wsj.com/us-news/how-minnesota-beca...
4,2026-01-09-22-50-09 +0000,nypost,Antisemitic vandal carves ‘Die Thieves’ on NYC...,https://nypost.com/2026/01/09/us-news/antisemi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
24,trump,59
20,ice,26
54,oil,21
59,venezuela,21
193,iran,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
333,2026-01-09-00-09-00 +0000,wsj,President Trump lashed out at five GOP senator...,https://www.wsj.com/politics/policy/senate-vot...,129
221,2026-01-09-12-23-44 +0000,nyt,Trump Indicates He Will Meet Venezuela’s Macha...,https://www.nytimes.com/2026/01/09/world/ameri...,126
271,2026-01-09-09-00-00 +0000,cbc,How Trump and Vance's accounts of Minneapolis ...,https://www.cbc.ca/news/world/minneapolis-ice-...,121
42,2026-01-09-22-06-42 +0000,nyt,Trump’s $100 Billion Venezuela Oil Plan Gets a...,https://www.nytimes.com/2026/01/09/business/en...,117
7,2026-01-09-22-46-43 +0000,nyt,Live Updates: Trump Pushes Oil Executives to I...,https://www.nytimes.com/live/2026/01/09/world/...,116


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
333,129,2026-01-09-00-09-00 +0000,wsj,President Trump lashed out at five GOP senator...,https://www.wsj.com/politics/policy/senate-vot...
90,101,2026-01-09-20-13-50 +0000,nypost,Minnesota schools allow remote learning amid I...,https://nypost.com/2026/01/09/us-news/minnesot...
289,58,2026-01-09-05-04-26 +0000,nypost,Russia fires nuclear-capable Oreshnik missile ...,https://nypost.com/2026/01/09/world-news/russi...
239,55,2026-01-09-11-47-00 +0000,wsj,Iran’s Supreme Leader Says He Won’t Bow Down t...,https://www.wsj.com/world/middle-east/irans-su...
122,51,2026-01-09-18-46-01 +0000,nypost,Sergey Brin joins fellow Google co-founder Lar...,https://nypost.com/2026/01/09/business/fellow-...
302,45,2026-01-09-03-08-27 +0000,nypost,Trump plans to meet Venezuelan opposition lead...,https://nypost.com/2026/01/08/us-news/trump-pl...
240,42,2026-01-09-11-29-22 +0000,nyt,"Wary of Investing in Venezuela, Big Oil Heads ...",https://www.nytimes.com/2026/01/09/business/en...
63,41,2026-01-09-21-28-00 +0000,wsj,American employers added a seasonally adjusted...,https://www.wsj.com/economy/jobs/jobs-report-d...
143,40,2026-01-09-17-42-06 +0000,bbc,Owner of Swiss ski bar held in custody after d...,https://www.bbc.com/news/articles/c4g0mq520zeo...
292,39,2026-01-09-04-44-22 +0000,cbc,Court records detail prior incident involving ...,https://www.cbc.ca/news/world/ice-officer-minn...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
